In [1]:
using JuMP
using CPLEX
#using GLPKMathProgInterface

## THE DIET PROBLEM (CLASSICAL LINEAR PROGRAMMING MODEL)

This model determines a least cost diet which meets the daily
allowances of nutrients for a moderately active man weighing 154 lbs.

We use the classical instance for the diet problem, proposed by:

Dantzig, G B, Chapter 27.1. In Linear Programming and Extensions. Princeton University Press, Princeton, New Jersey, 1963

Stigler's Nutrition model, the formulation is as follows:

$\min \sum_{j\in \mathbb{J}} c_j x_j$

s.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

In [59]:
A=[44.7  1411   2.0   365  0     55.4  33.3  441  0;
    36   897    1.7   99   30.9  17.4  7.9   106  0;
    8.4  422    15.1  9    26    3     23.5  11   60;
    20.6 17     0.6   6    55.8  0.2   0     0    0;
    7.4  448    16.4  19   28.1  0.8   10.3  4    0;
    15.7 661    1     48   0     9.6   8.1   471  0;
    41.7 0      0     0    0.2   0     0.5   5    0;
    2.2  333    0.2   139  169.2 6.4   50.8  316  525;
    4.4  249    0.3   37   0     18.2  3.6   79   0;
    5.8  705    6.8   45   3.5   1     4.9   209  0;
    2.4  138    3.7   80   69    4.3   5.8   37   862;
    2.6  125    4     36   7.2   9     4.5   26   5369;
    5.8  166    3.8   59   16.6  4.7   5.9   21   1184;
    14.3 336    1.8   118  6.7   29.4  7.1   198  2522;
    1.1  106    0     138  918.4 5.7   13.8  33   2755;
    9.6  138    2.7   54   290.7 8.4   5.4   83   1912;
    8.5  87     1.7   173  86.8  1.2   4.3   55   57;
    12.8 99     2.5   154  85.7  3.9   4.3   65   257;
    17.4 1055   3.7   459  5.1   26.9  38.2  93   0;
    26.9 1691   11.4  792  0     38.4  24.6  217  0]

A=A';

b=[30,70,0.8,12,5,1.8,2.7,18,75]

c=ones(20,1);

Creating a range for the coefficient costs

In [60]:
function randomcoef(c)
    cl=zeros(length(c));
    cu=zeros(length(c));
    for i=1:length(c)
        cl[i]=c[i]-rand()/2;
        cu[i]=c[i]+rand()/2;
    end
    return cl,cu
end
c_min,c_max=randomcoef(c)

([0.683203,0.533,0.879617,0.738942,0.630662,0.706235,0.852456,0.815745,0.96259,0.907289,0.671485,0.852908,0.908782,0.801274,0.549522,0.554193,0.65654,0.813833,0.594242,0.796726],[1.06407,1.45685,1.15635,1.28947,1.46319,1.40019,1.36906,1.30643,1.05472,1.38774,1.43437,1.15178,1.25712,1.38037,1.19996,1.11738,1.46727,1.1747,1.20637,1.09278])

From now on, we contruct our Min Max Regret model which is formulated as

$\min_{x} \max_{y,s} \sum_{j \in \mathbb{J}} c^s_j x_j - c^s_j y^s_j$

S.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$\sum_{j \in \mathbb{J}} a_{ij}y^s_j \ge b_i, \quad \forall i \in \mathbb{i}, \forall s \in \mathbb{S}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

$y^s_{j}\ge 0, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

$\underline{c}_j\le c^s_j \le \overline{c}_j, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

In [61]:
function solveLP(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

#Fixing some x
function solveLP(A,b,c,solver,v,max)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    @constraint(submodel, sum(v[j]*x[j] for j in 1:n)==max)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor,sol=getobjectivevalue(submodel),getvalue(x)
    return valor,sol
end

solveLP (generic function with 2 methods)

We'll calculate the upper bounds for $x_j$, for this purpose, a LP is solves fixing $x_j$ at its empirical maximum

$\hat{x}_j=\max_{i:a_{ij}>0}\Big\{\frac{b_j}{a_{ij}}\Big\},\qquad \forall ~j \in J$

In [62]:
xmax=zeros(length(c));
for j=1:length(c)
    v=zeros(length(c));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    pos=find(x->x==xm,b./A[:,j])
    pos1=find(x->x==xm,maxim)
    new=xm-Ar[pos]
    if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
        xmax[j]=new[1]
    else
        xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    end
end
println(xmax)

[0.636075,0.797014,3.57143,8.31458,4.1368,1.87576,24.4699,13.6364,6.78312,5.13609,12.5,11.5385,5.17241,2.0979,26.9118,3.125,3.52941,2.34375,1.68782,1.08018]


In [63]:
#Subproblem (NP-hard) function 
function SubproblemLP(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b)
    r=size(A,1)
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    ind=[]
    for j=1:n
        if fix1[j]==1
            setvalue(zup[j],1)
            setvalue(zun[j],0)
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]==1
                setvalue(zup[j],0)
                setvalue(zup[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            end
            push!(ind,j)
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], zup[j]*(xmax[j]-xbar[j]) >= yup[j])
    @constraint(m, minusy[j in ind], zun[j]*xbar[j] >= yun[j])
        
    status = solve(m)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]==1
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end    
    return valor,cost_scen
end

SubproblemLP (generic function with 1 method)

In [71]:
function solveRLP(A,b,cminus, cplus, xmax, n::Int64, solver=CplexSolver())
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    @constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)        
    #print(m)
    
    status = solve(m)
    #println(status)
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB==0
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    
    while conteo<=10
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]<=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\t",fixup,"\n")
        #println("\t\t",fixun,"\n")
        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b)
        
        #New cut
        rhs=solveLP(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        @constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs )
        
        print("-----------------######-------------------")
        print(m)
        print("-----------------######-------------------")
        
        status = solve(m)
        println(status)
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        if abs(LB-LBprime)<=tol
            conteo += 1
        else
            conteo = 1
        end
        
        it += 1
        print("GAP entre soluciones consecutivas", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

solveRLP (generic function with 2 methods)

In [72]:
solveRLP(A,b,c_min, c_max, xmax, length(c))

Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 7 columns.
Reduced LP has 9 rows, 14 columns, and 115 nonzeros.
Presolve time = 0.00 sec. (0.03 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            -0.015247
-----------------######-------------------Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9

-----------------######-------------------CPLEX Error  3003: Not a mixed-integer problem.
Optimal
GAP entre soluciones consecutivas0.0-----------------######-------------------Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9 x[3] + 6 x[4] + 19 x[5] + 48 x[6] + 139 x[8] + 37 x[9] + 45 x[10] + 80 x[11] + 36 x[12] + 59 x[13] + 118 x[14

Optimal
GAP entre soluciones consecutivas0.0-----------------######-------------------Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9 x[3] + 6 x[4] + 19 x[5] + 48 x[6] + 139 x[8] + 37 x[9] + 45 x[10] + 80 x[11] + 36 x[12] + 59 x[13] + 118 x[14] + 138 x[15] + 54 x[16] + 173 x[17] + 154 x[18] + 459 x[19] + 792 x[20] ≥ 12
 30.9 x[2] +

-----------------######-------------------CPLEX Error  3003: Not a mixed-integer problem.
Optimal
GAP entre soluciones consecutivas0.0-----------------######-------------------Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9 x[3] + 6 x[4] + 19 x[5] + 48 x[6] + 139 x[8] + 37 x[9] + 45 x[10] + 80 x[11] + 36 x[12] + 59 x[13] + 118 x[14

-----------------######-------------------CPLEX Error  3003: Not a mixed-integer problem.
Optimal
GAP entre soluciones consecutivas0.0-----------------######-------------------Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9 x[3] + 6 x[4] + 19 x[5] + 48 x[6] + 139 x[8] + 37 x[9] + 45 x[10] + 80 x[11] + 36 x[12] + 59 x[13] + 118 x[14

In [58]:
#Modelo completo
A=[0.1 0; 0 0.1; 0.1 0.2; 0.2 0.1]
b=[0.4, 0.6, 2, 1.7]
c=[0.2,0.08]
c_min=[0.1,0.07]
c_max=[0.3,0.09]

y1=solveLP(A,b,[0.1,0.07],CplexSolver(CPX_PARAM_SCRIND=0))
println("y1 opt: ", y1)
y2=solveLP(A,b,[0.1,0.09],CplexSolver(CPX_PARAM_SCRIND=0))
println("y2 opt: ", y2)
y3=solveLP(A,b,[0.3,0.07],CplexSolver(CPX_PARAM_SCRIND=0))
println("y3 opt: ", y3)
y4=solveLP(A,b,[0.3,0.09],CplexSolver(CPX_PARAM_SCRIND=0))
println("y4 opt: ", y4)

M=Model(solver=CplexSolver(CPX_PARAM_SCRIND=0))

@variable(M, x[1:2]>=0)
@variable(M, t)

@objective(M, Min, t)

@constraint(M, req[i = 1:4], sum(A[i,j]*x[j] for j in 1:2) >= b[i])
@constraint(M, t >= 0.1*x[1]+0.07*x[2]-y1)
@constraint(M, t >= 0.1*x[1]+0.09*x[2]-y2)
@constraint(M, t >= 0.3*x[1]+0.07*x[2]-y3)
@constraint(M, t >= 0.3*x[1]+0.09*x[2]-y4)

print(M)

status = solve(M)
println(getobjectivevalue(M))
println(getvalue(x))


Min 0.1 x[1] + 0.07 x[2]
Subject to
 0.1 x[1] ≥ 0.4
 0.1 x[2] ≥ 0.6
 0.1 x[1] + 0.2 x[2] ≥ 2
 0.2 x[1] + 0.1 x[2] ≥ 1.7
 x[i] ≥ 0 ∀ i ∈ {1,2}
y1 opt: 1.0033333333333334
Min 0.1 x[1] + 0.09 x[2]
Subject to
 0.1 x[1] ≥ 0.4
 0.1 x[2] ≥ 0.6
 0.1 x[1] + 0.2 x[2] ≥ 2
 0.2 x[1] + 0.1 x[2] ≥ 1.7
 x[i] ≥ 0 ∀ i ∈ {1,2}
y2 opt: 1.1566666666666667
Min 0.3 x[1] + 0.07 x[2]
Subject to
 0.1 x[1] ≥ 0.4
 0.1 x[2] ≥ 0.6
 0.1 x[1] + 0.2 x[2] ≥ 2
 0.2 x[1] + 0.1 x[2] ≥ 1.7
 x[i] ≥ 0 ∀ i ∈ {1,2}
y3 opt: 1.83
Min 0.3 x[1] + 0.09 x[2]
Subject to
 0.1 x[1] ≥ 0.4
 0.1 x[2] ≥ 0.6
 0.1 x[1] + 0.2 x[2] ≥ 2
 0.2 x[1] + 0.1 x[2] ≥ 1.7
 x[i] ≥ 0 ∀ i ∈ {1,2}
y4 opt: 2.01
Min t
Subject to
 0.1 x[1] ≥ 0.4
 0.1 x[2] ≥ 0.6
 0.1 x[1] + 0.2 x[2] ≥ 2
 0.2 x[1] + 0.1 x[2] ≥ 1.7
 t - 0.1 x[1] - 0.07 x[2] ≥ -1.0033333333333334
 t - 0.1 x[1] - 0.09 x[2] ≥ -1.1566666666666667
 t - 0.3 x[1] - 0.07 x[2] ≥ -1.83
 t - 0.3 x[1] - 0.09 x[2] ≥ -2.01
 x[i] ≥ 0 ∀ i ∈ {1,2}
 t
0.05575555555555534
[4.22222,8.55556]
